In [1]:
import pandas as pd
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors
import numpy as np

In [2]:
df_datos_demo = pd.read_csv("C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos\\datos_demograficos.csv")
df_datos_generacion = pd.read_csv("C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos\\datos_generacion.csv")
df_datos_PIB = pd.read_csv("C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos\\datos_PIB.csv")
df_anio = pd.read_csv("C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos\\df_anio.csv")
df_ccaa= pd.read_csv("C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos\\df_ccaa.csv")
df_demanda = pd.read_csv("C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos\\df_demanda.csv")
df_edades = pd.read_csv("C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos\\df_edades.csv")
df_nac_extran = pd.read_csv("C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos\\df_nac_extran.csv")
df_provincias = pd.read_csv("C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos\\df_provincias.csv")
df_sexos = pd.read_csv("C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos\\df_sexos.csv")
df_tipo_generacion = pd.read_csv("C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos\\df_tipo_generacion.csv")

In [19]:
df_datos_generacion.head()

,Energía (MWh),Porcentage de Generación Renovable,COD-CCAA,Mes,Tipo de energía,ID_Año
0,25288.605,0.029517,4,1,1,1
1,33933.350,0.030779,4,2,1,1
2,35913.000,0.029645,4,3,1,1
3,44050.848,0.041340,4,4,1,1
4,88535.452,0.068497,4,5,1,1


In [20]:
df_datos_generacion.dtypes

Energía (MWh)                         float64
Porcentage de Generación Renovable    float64
COD-CCAA                                int64
Mes                                     int64
Tipo de energía                         int64
ID_Año                                  int64
dtype: object

In [4]:
df_datos_demo = df_datos_demo.merge(df_anio, how='inner', left_on= 'Año', right_on= 'Año').drop(columns='Año')
df_datos_generacion = df_datos_generacion.merge(df_anio, how='inner', left_on= 'Año', right_on= 'Año').drop(columns='Año')
df_datos_PIB = df_datos_PIB.merge(df_anio, how='inner', left_on= 'Año', right_on= 'Año').drop(columns='Año')
df_demanda = df_demanda.merge(df_anio, how='inner', left_on= 'Año', right_on= 'Año').drop(columns='Año')

In [6]:
try:
    conexion = psycopg2.connect(
    database = 'ETL',
    user = 'postgres',
    password = 'admin',
    host = 'localhost',
    port = '5432')
except OperationalError as e: #el alias es e
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print('la contraseña es erronea')
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print('error de conexion')
    else:
        print(f'el error es el siguiente {e}')

In [7]:
cursor = conexion.cursor() #para crear la tabla desde aquí
query_creacion_CCAA = """ 
    CREATE TABLE IF NOT EXISTS ccaa (
    COD_CCAA int PRIMARY KEY,
    CCAA varchar(100));

"""

cursor.execute(query_creacion_CCAA)
conexion.commit()

In [8]:
cursor = conexion.cursor() #para crear la tabla desde aquí
query_creacion_type_gen = """ 
    CREATE TABLE IF NOT EXISTS tipo_generacion (
    id_tipo_generacion int PRIMARY KEY,
    tipo_generacion varchar(100));

"""

cursor.execute(query_creacion_type_gen)
conexion.commit()

In [9]:
cursor = conexion.cursor() #para crear la tabla desde aquí
query_creacion_sexo = """ 
    CREATE TABLE IF NOT EXISTS sexo (
    id_sexo int PRIMARY KEY,
    Sexo varchar(100));

"""

cursor.execute(query_creacion_sexo)
conexion.commit()

In [10]:
cursor = conexion.cursor() #para crear la tabla desde aquí
query_creacion_grupo_edades = """ 
    CREATE TABLE IF NOT EXISTS edad (
    id_grupo_edad int PRIMARY KEY,
    grupo_edad varchar(100));

"""

cursor.execute(query_creacion_grupo_edades)
conexion.commit()

In [11]:
cursor = conexion.cursor() #para crear la tabla desde aquí
query_creacion_grupo_nac_extr = """ 
    CREATE TABLE IF NOT EXISTS nacionales_extranjeros (
    id_nacional_extranjero int PRIMARY KEY,
    categoria varchar(100));

"""

cursor.execute(query_creacion_grupo_nac_extr)
conexion.commit()

In [12]:
cursor = conexion.cursor() #para crear la tabla desde aquí
query_creacion_provincias = """ 
    CREATE TABLE IF NOT EXISTS provincias (
    id_provincia int PRIMARY KEY,
    provincia varchar(100));

"""

cursor.execute(query_creacion_provincias)
conexion.commit()

In [14]:
cursor = conexion.cursor() #para crear la tabla desde aquí
query_creacion_anio = """ 
    CREATE TABLE IF NOT EXISTS anio (
    id_anio int PRIMARY KEY,
    anio int);

"""

cursor.execute(query_creacion_anio)
conexion.commit()

In [ ]:
cursor = conexion.cursor() #para crear la tabla desde aquí
query_creacion_d_demo = """ 
    CREATE TABLE IF NOT EXISTS datos_demograficos (
    total int,
    id_provincia int, 
    id_grupo_edad int,
    id_nacional_extranjero int,
    id_sexo int, 
    id_anio int,     
    CONSTRAINT fk_provincias FOREIGN KEY (id_provincia)
        REFERENCES provincias(id_provincia),
    CONSTRAINT fk_edad FOREIGN KEY (id_grupo_edad)
        REFERENCES edad(id_grupo_edad),
    CONSTRAINT fk_nacionales_extranjeros FOREIGN KEY (id_nacional_extranjero)
        REFERENCES nacionales_extranjeros(id_nacional_extranjero),
    CONSTRAINT fk_sexo FOREIGN KEY (id_sexo)
        REFERENCES sexo(id_sexo),
    CONSTRAINT fk_anio FOREIGN KEY (id_anio)
        REFERENCES anio(id_anio)

);

"""

cursor.execute(query_creacion_d_demo)
conexion.commit()

In [ ]:
cursor = conexion.cursor() #para crear la tabla desde aquí
query_creacion_pib = """ 
    CREATE TABLE IF NOT EXISTS pib (
    ramas_actividad varchar(100),
    total int, 
    id_provincia int,
    id_anio int,     
    CONSTRAINT fk_provincias FOREIGN KEY (id_provincia)
        REFERENCES provincias(id_provincia),
    CONSTRAINT fk_anio FOREIGN KEY (id_anio)
        REFERENCES anio(id_anio)

);

"""

cursor.execute(query_creacion_pib)
conexion.commit()

In [ ]:
cursor = conexion.cursor() #para crear la tabla desde aquí
query_creacion_generacion = """ 
    CREATE TABLE IF NOT EXISTS generacion (
    energia (mwh) float,
    porcentage_generacion int, 
    COD_CCAA int,
    mes int,
    id_tipo_generacion int, 
    id_anio int,     
    CONSTRAINT fk_COD_CCAA FOREIGN KEY (COD_CCAA)
        REFERENCES COD_CCAA(COD_CCAA),
    CONSTRAINT fk_tipo_generacion FOREIGN KEY (id_tipo_generacion)
        REFERENCES tipo_generacion(id_tipo_generacion),
    CONSTRAINT fk_anio FOREIGN KEY (id_anio)
        REFERENCES anio(id_anio)

);

"""

cursor.execute(query_creacion_generacion)
conexion.commit()

In [ ]:
cursor = conexion.cursor() #para crear la tabla desde aquí
query_creacion_generacion = """ 
    CREATE TABLE IF NOT EXISTS generacion (
    energia (mwh) float,
    porcentage_generacion int, 
    COD_CCAA int,
    mes int,
    id_tipo_generacion int, 
    id_anio int,     
    CONSTRAINT fk_COD_CCAA FOREIGN KEY (COD_CCAA)
        REFERENCES COD_CCAA(COD_CCAA),
    CONSTRAINT fk_tipo_generacion FOREIGN KEY (id_tipo_generacion)
        REFERENCES tipo_generacion(id_tipo_generacion),
    CONSTRAINT fk_anio FOREIGN KEY (id_anio)
        REFERENCES anio(id_anio)

);

"""

cursor.execute(query_creacion_generacion)
conexion.commit()